In [2]:
import os
import gc
from joblib import dump, load, Parallel, delayed
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb

In [3]:
mode = "train"

In [4]:
df = pd.read_parquet("../input/{}_full_features.parquet".format(mode))

In [6]:
exclude_features = []

exclude_features += [
    "customer_ID", 
    "target",
]

features = [col for col in df.columns if col not in exclude_features]

cat_features_base = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
] 
cat_features = []
for feature in features:
    for cat_feature_base in cat_features_base:
        if cat_feature_base in feature:
            cat_features.append(feature)
            
target = "target"

In [7]:
base_stat_agg_group_1_to_del = load("base_stat_agg_group_1_to_del.pkl")[0]
base_stat_agg_group_2_to_del = load("base_stat_agg_group_2_to_del.pkl")[0]
base_stat_agg_to_del = base_stat_agg_group_1_to_del + base_stat_agg_group_2_to_del

sma_agg_to_del = load("sma_agg_to_del.pkl")[0]

# quantile_agg_to_del = load("quantile_agg_to_del.pkl")[0]
quantile_agg_to_del = [feature for feature in features if "quantile" in feature]

# skew_kurtosis_agg_to_del = load("skew_kurtosis_agg_to_del.pkl")[0]
skew_kurtosis_agg_to_del = [feature for feature in features if "skew" in feature and "realized" not in feature] + \
    [feature for feature in features if "kurtosis" in feature]

mean_diff_max_drawdown_agg_to_del = load("mean_diff_max_drawdown_agg_to_del.pkl")[0]

# realized_volatility_abs_skew_agg_to_del = load("realized_volatility_abs_skew_agg_to_del.pkl")[0]
realized_volatility_abs_skew_agg_to_del = [feature for feature in features if "realized_volatility" in feature] + \
    [feature for feature in features if "realized_abs_skew" in feature]

features = [feature for feature in features if feature not in (
    base_stat_agg_to_del + \
    sma_agg_to_del + \
    quantile_agg_to_del + \
    skew_kurtosis_agg_to_del + \
    mean_diff_max_drawdown_agg_to_del + \
    realized_volatility_abs_skew_agg_to_del
)]

In [8]:
len(features)

1065

In [9]:
if mode == "train":
    all_columns = ["customer_ID"] + features + ["target"]
else:
    all_columns = ["customer_ID"] + feature

In [10]:
df = df[all_columns]

In [11]:
df.to_parquet("../input/{}_final_features.parquet".format(mode))